## Access MySQL from python

In [1]:
#!/usr/bin/python

import   pandas          as pd
import   MySQLdb
import   mysql.connector
from     sqlalchemy      import create_engine

# Open database connection
db = MySQLdb.connect("localhost","root","","matrimony" )

# prepare a cursor object using cursor() method
cursor = db.cursor()

# execute SQL query using execute() method.
cursor.execute("SELECT VERSION()")

# Fetch a single row using fetchone() method.
db_version = cursor.fetchone()
print("Database version : %s " % db_version)

# disconnect from server
db.close()

Database version : 10.1.31-MariaDB 


### Access data from mysql table -  database_for_matrimony1

In [2]:
import   mysql.connector, re
import   pandas              as  pd
import   numpy               as  np

mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  passwd="",
  database="matrimony"
)

mycursor = mydb.cursor()

mycursor.execute("SELECT * FROM database_for_matrimony1")

myresult = mycursor.fetchall()

mycol = ['S_No','Profile_ID','Age','Gender','Marital_Status','Caste',\
          'Sub_Caste','State','City','Country','Height','physical_status',\
          'Family_Status','Family_Type','Family_Values','Groom_s_Age',\
          'Groom_s_Education','Groom_s_Income','Groom_s_Location',\
          'About_Description']     


df      = pd.DataFrame(myresult, columns= mycol)

print(df.head())  
print(list(df))
print(df.info())

   S_No Profile_ID  Age  Gender Marital_Status     Caste    Sub_Caste  \
0     1   M5929711   27  Female  Never married  Mudaliar    Melakarar   
1     2   M5929712   28  Female  Never married  Mudaliar    Melakarar   
2     3   M5929713   28  Female  Never married     Nayar      Nambiar   
3     4   M5929714   27  Female        widower     Nayar      Nambiar   
4     5   M5929715   27  Female        widower    Kallar  Mukkulathor   

        State     City Country Height physical_status       Family_Status  \
0  Tamil Nadu  Chennai   India    5.8          Normal        Middle Class   
1  Tamil Nadu  Chennai   India    5.8          Normal  Upper Middle Class   
2  Tamil Nadu  Chennai   India    5.6          Normal                Rich   
3  Tamil Nadu  Chennai   India    5.9          Normal            Affluent   
4  Tamil Nadu  Chennai   India    5.8          Normal                Rich   

  Family_Type Family_Values Groom_s_Age Groom_s_Education  Groom_s_Income  \
0       Joint      Or

In [3]:
n   =  df.shape[0] # number of rows

Pref_Age_Lo = []
Pref_Age_Up = []

for i in range(n):
    if len(df['Groom_s_Age'][i]) == 5:
        lo, up = re.findall('\d+', df['Groom_s_Age'][i])
    else:
        lo     = df['Age'][i] + 1
        up     = df['Age'][i] + 5
        
    Pref_Age_Lo.append(lo)
    Pref_Age_Up.append(up)

Lower  = pd.Series(Pref_Age_Lo)
Upper  = pd.Series(Pref_Age_Up)

df['Partner_Age_LI']   =   Lower.values
df['Partner_Age_UI']   =   Upper.values

df['Partner_Age_LI'] = df['Partner_Age_LI'].astype('int32')
df['Partner_Age_UI'] = df['Partner_Age_UI'].astype('int32')


print(df.head(3).T)


                               0                   1              2
S_No                           1                   2              3
Profile_ID              M5929711            M5929712       M5929713
Age                           27                  28             28
Gender                    Female              Female         Female
Marital_Status     Never married       Never married  Never married
Caste                   Mudaliar            Mudaliar          Nayar
Sub_Caste              Melakarar           Melakarar        Nambiar
State                 Tamil Nadu          Tamil Nadu     Tamil Nadu
City                     Chennai             Chennai        Chennai
Country                    India               India          India
Height                       5.8                 5.8            5.6
physical_status           Normal              Normal         Normal
Family_Status       Middle Class  Upper Middle Class           Rich
Family_Type                Joint             Nuc

In [4]:
male_df        = df.loc[df['Gender'] == 'Male',['Profile_ID','Age','Gender','Marital_Status','Caste','Sub_Caste','Partner_Age_LI','Partner_Age_UI']]
female_df      = df.loc[df['Gender'] == 'Female',['Profile_ID','Age','Gender','Marital_Status','Caste','Sub_Caste','Partner_Age_LI','Partner_Age_UI']]

print(male_df.shape)
print(male_df.head())
print(female_df.shape)
print(female_df.head(13))


(52, 8)
   Profile_ID  Age Gender    Marital_Status    Caste    Sub_Caste  \
12   M5929723   27   Male  Awaiting Divorce   Thevar  Mukkulathor   
13   M5929724   30   Male          Divorced  Maravar  Mukkulathor   
14   M5929725   30   Male          Divorced   Kallar  Mukkulathor   
15   M5929726   24   Male     Never married    Nayar      Nambiar   
16   M5929727   24   Male     Never married  Maravar  Mukkulathor   

    Partner_Age_LI  Partner_Age_UI  
12              25              27  
13              28              29  
14              28              29  
15              22              23  
16              22              23  
(48, 8)
   Profile_ID  Age  Gender Marital_Status     Caste    Sub_Caste  \
0    M5929711   27  Female  Never married  Mudaliar    Melakarar   
1    M5929712   28  Female  Never married  Mudaliar    Melakarar   
2    M5929713   28  Female  Never married     Nayar      Nambiar   
3    M5929714   27  Female        widower     Nayar      Nambiar   
4    M5

In [5]:
df_col = list(female_df)
print(df_col)

['Profile_ID', 'Age', 'Gender', 'Marital_Status', 'Caste', 'Sub_Caste', 'Partner_Age_LI', 'Partner_Age_UI']


## Create corpus data from the user search results

In [6]:
file1 = open("Mate_Corpus.txt","w") 
L = ["This corpus is used to assist the users to get their potential matches\n"]  
 
file1.writelines(L) 



In [7]:
n   =  female_df.shape[0] # number of rows
n   =  10
for i in range(n):    
    user_age        = female_df['Age'][i]
    user_profile_ID = female_df['Profile_ID'][i]

    ### Female preference

    pref_age_l      = female_df['Partner_Age_LI'][i]                                
    pref_age_u      = female_df['Partner_Age_UI'][i]

    msg = "\n Record: %d Profile_ID %s User Age: %d preferred partner age: Between %d and %d" \
           % (i, user_profile_ID, user_age, int(pref_age_l), int(pref_age_u))
    print(msg)
    
    L1     = msg    user_age        = female_df['Age'][i]
    user_profile_ID = female_df['Profile_ID'][i]

    ### Female preference

    pref_age_l      = female_df['Partner_Age_LI'][i]                                
    pref_age_u      = female_df['Partner_Age_UI'][i]

    msg = "\n Record: %d Profile_ID %s User Age: %d preferred partner age: Between %d and %d" \
           % (i, user_profile_ID, user_age, int(pref_age_l), int(pref_age_u))
    print(msg)
    
    L1     = msg
    file1.writelines(L1) 
    
    print(male_df['Partner_Age_LI'].dtype)
    pref_male_df   = male_df.loc[(male_df['Age'] >=  int(pref_age_l)) & (male_df['Age'] <=  int(pref_age_u)) &\
                                 (male_df['Partner_Age_LI'] <=  int(user_age)) & (male_df['Partner_Age_UI'] >=  int(user_age))]

    print(pref_male_df.head(3).T)
    
    msg = ' Count of matches: %d' % (pref_male_df['Profile_ID'].count())
    
    print(msg)

    file1.writelines(L1) 
    
    print(male_df['Partner_Age_LI'].dtype)
    pref_male_df   = male_df.loc[(male_df['Age'] >=  int(pref_age_l)) & (male_df['Age'] <=  int(pref_age_u)) &\
                                 (male_df['Partner_Age_LI'] <=  int(user_age)) & (male_df['Partner_Age_UI'] >=  int(user_age))]

    print(pref_male_df.head(3).T)
    
    msg = ' Count of matches: %d' % (pref_male_df['Profile_ID'].count())
    
    print(msg)
    
    L2     = msg
    file1.writelines(L2) 

file1.close() #to change file access modes 


 Record: 0 Profile_ID M5929711 User Age: 27 preferred partner age: Between 28 and 30
int32
                         17           18           19
Profile_ID         M5929728     M5929729     M5929730
Age                      29           29           30
Gender                 Male         Male         Male
Marital_Status     Divorced     Divorced     Divorced
Caste                Thevar       Kallar      Maravar
Sub_Caste       Mukkulathor  Mukkulathor  Mukkulathor
Partner_Age_LI           27           27           25
Partner_Age_UI           29           29           29
 Count of matches: 11

 Record: 1 Profile_ID M5929712 User Age: 28 preferred partner age: Between 29 and 30
int32
                         13           14           17
Profile_ID         M5929724     M5929725     M5929728
Age                      30           30           29
Gender                 Male         Male         Male
Marital_Status     Divorced     Divorced     Divorced
Caste               Maravar       Kall